In [ ]:
# 1. وارد کردن کتابخانه‌ها
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# 2. تنظیمات پایه
IMG_SIZE = 96
BATCH_SIZE = 32
EPOCHS = 15

# 3. بارگذاری دیتاست CIFAR-10 و پیش‌پردازش
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Resize تصاویر و نرمال‌سازی
x_train = tf.image.resize(x_train, (IMG_SIZE, IMG_SIZE)) / 255.0
x_test = tf.image.resize(x_test, (IMG_SIZE, IMG_SIZE)) / 255.0

# تبدیل برچسب‌ها به one-hot
y_train_cat = tf.keras.utils.to_categorical(y_train, 10)
y_test_cat = tf.keras.utils.to_categorical(y_test, 10)

# 4. ساخت مدل MobileNetV2 با وزن‌های pretrained و اضافه کردن لایه‌های نهایی
base_model = MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                         include_top=False,
                         weights='imagenet')
base_model.trainable = False  # قفل کردن لایه‌های pretrained در ابتدا

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
outputs = Dense(10, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=outputs)

# 5. کامپایل مدل
model.compile(optimizer=Adam(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# 6. آموزش مدل بدون Data Augmentation
history = model.fit(x_train, y_train_cat,
                    validation_data=(x_test, y_test_cat),
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS)

# 7. رسم نمودار دقت و خطا
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Accuracy')

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss')

plt.show()

# 8. ارزیابی مدل با classification report و confusion matrix
y_pred_probs = model.predict(x_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = y_test.flatten()

print("Classification Report:\n")
print(classification_report(y_true, y_pred, target_names=[
    'airplane', 'automobile', 'bird', 'cat', 'deer',
    'dog', 'frog', 'horse', 'ship', 'truck']))

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10,8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['airplane', 'automobile', 'bird', 'cat', 'deer',
                         'dog', 'frog', 'horse', 'ship', 'truck'],
            yticklabels=['airplane', 'automobile', 'bird', 'cat', 'deer',
                         'dog', 'frog', 'horse', 'ship', 'truck'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()
